In [2]:
from gridEnv import Grid;
import numpy as np;

In [5]:
from IPython.display import clear_output
from time import sleep;
g = Grid(length=4,width=6,start=(0,0),terminals=[(3,5)]);
for i in range(4):
    g.display()
    print (g.step(3))
    sleep(1);
    clear_output(wait=True)


|. . . . . .|
|. . . . . .|
|. . . . . .|
|C . . . . T|
(True, -1)


# Single Episode

In [6]:
hasEnded = False;
g.reset()
while(hasEnded == False):
    g.display()
    print (g.step(0,update=False),g.step(1,update=False),g.step(2,update=False),g.step(3,update=False))
    hasEnded,reward = g.step(np.random.randint(0,4))
    sleep(0.2);
    clear_output(wait=True)
g.display()
print ('Loop Ended')


|C . . . . .|
|. . . . . .|
|. . . . . .|
|. . . . . T|
Loop Ended


# SARSA Update

In [8]:
gamma = 0.8;
alpha = 0.3;
for i in range(1000):
    g.display()
    g.display_policy()
    
    #Getting Current Action
    s = g.get_state()
    q_s = g.get_q_value(s);
    best_a = np.argmax(q_s)
    a = g.greedy_sample(best_a)
    
    hasEnded,r = g.step(a) # Moving a step ahead
    
    #Getting next action
    snext = g.get_state();
    qnext = g.get_q_value(snext)
    best_anext = np.argmax(qnext);
    anext = g.greedy_sample(best_anext)
    
    #SARSA UPDATE
    q_s[a] = alpha*q_s[a] + (1-alpha)*(r+gamma*qnext[best_anext])
    g.set_q_value(s,q_s)

    sleep(0.05);
    clear_output(wait=True)
    

|. . . . . .|
|. . . . . .|
|. . . . . .|
|C . . . . T|
|→ → ↓ → ↓ ←|
|↑ → → ↓ ↓ ↓|
|→ → → ↓ ↓ ↓|
|→ → → → → ←|


# Monte Carlo REINFORCE

In [9]:
gamma = 0.7;
alpha = 0.3;
g.qvalue=g.qvalue*0.01
has_ended = False
for i in range(100):
    sar_pairs=[];
    while(has_ended == False):
        g.display()
        g.display_policy()
        #Getting Current Action
        s = g.get_state()
        q_s = g.get_q_value(s);
        best_a = np.argmax(q_s)
        a = g.greedy_sample(best_a,epsilon=0.7)
        
        has_ended,r = g.step(a) # Moving a step ahead
        sar_pairs.append({"s":s,"a":a,"r":r});
        sleep(0.1);
        clear_output(wait=True)
    has_ended=False;
    discounted_r=0;
    sar_pairs.reverse() # Reversing the pairs
    for i,sar in enumerate(sar_pairs):
        if i == 0 :
            discounted_r = sar["r"]
        else:
            discounted_r = sar["r"]+gamma*discounted_r
            q_s = g.get_q_value(sar["s"]);
            a = sar["a"]
            q_s[a] = alpha*q_s[a] + (1-alpha)*(discounted_r)
            g.set_q_value(s,q_s)
    

|C . . . . .|
|. . . . . .|
|. . . . . .|
|. . . . . T|
|← ← ← → ← ←|
|← → → ↓ ↓ ↓|
|→ → → ↓ ↓ ↓|
|← → → → → ←|
